In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imutils

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
image_path = os.path.join(DATA_FOLDER, 'keparoi_left_frame.jpg')
right_image_path = os.path.join(DATA_FOLDER, 'keparoi_right_frame.jpg')

In [ ]:
image = cv2.imread(image_path)
right_image = cv2.imread(right_image_path)

In [ ]:
plt.imshow(image)

In [ ]:
camera_matrix = np.load(os.path.join(DATA_FOLDER, 'camera_matrix.npy'))
distortions = np.load(os.path.join(DATA_FOLDER, 'camera_distortion.npy'))
undistorted_camera_matrix = np.load(os.path.join(DATA_FOLDER, 'undistorted_camera_matrix.npy'))

In [ ]:
camera_matrix

In [ ]:
distortions

In [ ]:
undistorted_camera_matrix

In [ ]:
h, w = image.shape[0:2]

new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, distortions, (w,h), 1, (w,h))
undistorted_image = cv2.undistort(image, camera_matrix, distortions, None, new_camera_matrix)

In [ ]:
plt.imshow(undistorted_image)

In [ ]:
mapx, mapy = cv2.initUndistortRectifyMap(camera_matrix, distortions, None, new_camera_matrix, (w,h), 5)
undistorted_image = cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)
x, y, w, h = roi
undistorted_image = undistorted_image[y:y+h, x:x+w]
plt.imshow(undistorted_image)

In [ ]:
def unwrap(imgIn, Cb, offset):

    img = imgIn

    #MAPPING
    def buildMap(Wd, Hd, R, Cx, Cy):
        map_x = np.zeros((Hd, Wd), np.float32)
        map_y = np.zeros((Hd, Wd), np.float32)
        for y in range(0, int(Hd - 1)):
            for x in range(0, int(Wd - 1)):
                r = (float(y) / float(Hd)) * R
                theta = (float(x-offset) / float(Wd)) * 2.0 * np.pi
                xS = Cx + r * np.sin(theta)
                yS = Cy + r * np.cos(theta)
                map_x.itemset((y, x), int(xS))
                map_y.itemset((y, x), int(yS))

        return map_x, map_y

    #UNWARP
    def unwarp(img, xmap, ymap):
        output = cv2.remap(img, xmap, ymap, cv2.INTER_LINEAR)
        return output

    #IMAGE CENTER
    Cx = img.shape[0]/2
    Cy = img.shape[1]/2

    #RADIUS OUTER
    R = Cb - Cx

    #DESTINATION IMAGE SIZE
    Wd = int(abs(2.0 * (R / 2) * np.pi))
    Hd = int(abs(R))

    #BUILD MAP
    xmap, ymap = buildMap(Wd, Hd, R, Cx, Cy)

    #UNWARP
    result = unwarp(img, xmap, ymap)

    return result

In [ ]:
unwarped = unwrap(image, 0, 0)

In [ ]:
def cylindricalWarp(img, K):
    """This function returns the cylindrical warp for a given image and intrinsics matrix K"""
    h_,w_ = img.shape[:2]
    # pixel coordinates
    y_i, x_i = np.indices((h_,w_))
    X = np.stack([x_i,y_i,np.ones_like(x_i)],axis=-1).reshape(h_*w_,3) # to homog
    Kinv = np.linalg.inv(K) 
    X = Kinv.dot(X.T).T # normalized coords
    # calculate cylindrical coords (sin\theta, h, cos\theta)
    A = np.stack([np.sin(X[:,0]),X[:,1],np.cos(X[:,0])],axis=-1).reshape(w_*h_,3)
    B = K.dot(A.T).T # project back to image-pixels plane
    # back from homog coords
    B = B[:,:-1] / B[:,[-1]]
    # make sure warp coords only within image bounds
    B[(B[:,0] < 0) | (B[:,0] >= w_) | (B[:,1] < 0) | (B[:,1] >= h_)] = -1
    B = B.reshape(h_,w_,-1)
    
    img_rgba = cv2.cvtColor(img,cv2.COLOR_BGR2BGRA) # for transparent borders...
    # warp the image according to cylindrical coords
    return cv2.remap(img_rgba, B[:,:,0].astype(np.float32), B[:,:,1].astype(np.float32), cv2.INTER_AREA, borderMode=cv2.BORDER_TRANSPARENT)

In [ ]:
img_cyl = cylindricalWarp(image, camera_matrix)
plt.imshow(img_cyl)

In [ ]:
img_cyl_left = cylindricalWarp(image, camera_matrix)
img_cyl_right = cylindricalWarp(right_image, camera_matrix)

fig, axs = plt.subplots(1, 2, figsize=(12, 12))
axs = axs.flatten()
for img, ax in zip([img_cyl_left, img_cyl_right], axs):
    ax.imshow(img)
    ax.axis('off')
fig.tight_layout()
plt.show()

In [ ]:
img_cyl_left = img_cyl_left[:,:,:3]
img_cyl_right = imutils.rotate(img_cyl_right[:,:,:3], -20)

In [ ]:
plt.imshow(img_cyl_right)

In [ ]:
img_cyl_left.shape

In [ ]:
stitcher = cv2.Stitcher_create(cv2.Stitcher_PANORAMA)
(status, stitched) = stitcher.stitch(images=[img_cyl_left, img_cyl_right])

In [ ]:
print(status)

In [ ]:
plt.imshow(stitched)
plt.show()